In [1]:
import sys
sys.path.extend(['..', '../', '../..', '../../' ])
%load_ext autoreload
%autoreload 2

# LeNet5 - MNIST

In [2]:
import paths
import keras
from src.parameters import metrics
from src.datasets.dataset_generators import DatasetGenerators
from src.keras_callback.time_history import TimeHistory
from src.export.csv_export import export_learning_history_to_csv, export_learning_validation_result_to_csv
from src.export.model_export import clear_exported_models
from src.keras_model.lenet5_network import LeNet5Network

Using TensorFlow backend.


In [3]:
network_name = 'lenet5'
fit_epochs = 50
dataset = 'mnist'
dataset_percentage = '5'

In [4]:
network = LeNet5Network()

In [5]:
dataset_generators = DatasetGenerators(
    name=f'{dataset}_{dataset_percentage}',
    dataset_dir=paths.data_location,
    image_size=network.image_size,
    batch_size=network.batch_size,
    grayscale=network.grayscale
)

Loading train set for mnist_5...
Found 2419 images belonging to 10 classes.
Loading test set for mnist_5...
Found 695 images belonging to 10 classes.
Loading validation set for mnist_5...
Found 349 images belonging to 10 classes.


In [6]:
model = network.get_compiled_model(
    classes=dataset_generators.test.num_classes
)

In [7]:
checkpoint_path = f'{paths.models_location}{network_name}/{dataset}/'
best_model_path = f'{checkpoint_path}best.h5'
clear_exported_models(checkpoint_path)

checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=f'{checkpoint_path}{{epoch:02d}}-{{val_accuracy}}.h5',
    save_best_only=False,
    save_weights_only=False
)

best_callback = keras.callbacks.ModelCheckpoint(
    filepath=best_model_path,
    save_best_only=True,
    verbose=1,
    save_weights_only=False
)

time_callback = TimeHistory()

model_callbacks = [checkpoint_callback, best_callback, time_callback]

In [8]:
fit_result = model.fit_generator(
    generator=dataset_generators.train,
    epochs=fit_epochs,
    validation_data=dataset_generators.test,
    callbacks = model_callbacks
)

Epoch 1/50
76/76 [==============================] - 2s 29ms/step - loss: 1.5420 - accuracy: 0.6742 - precision: 0.8343 - recall: 0.8352 - val_loss: 0.4050 - val_accuracy: 0.8734 - val_precision: 0.9909 - val_recall: 0.9364

Epoch 00001: val_loss improved from inf to 0.40502, saving model to /Users/igor/Python/CNN-Architecture-Comparison/models/lenet5/mnist/best.h5
Epoch 2/50
76/76 [==============================] - 1s 14ms/step - loss: 0.2953 - accuracy: 0.9124 - precision: 0.9461 - recall: 0.9476 - val_loss: 0.2314 - val_accuracy: 0.9165 - val_precision: 0.9545 - val_recall: 0.8890

Epoch 00002: val_loss improved from 0.40502 to 0.23142, saving model to /Users/igor/Python/CNN-Architecture-Comparison/models/lenet5/mnist/best.h5
Epoch 3/50
76/76 [==============================] - 1s 15ms/step - loss: 0.1779 - accuracy: 0.9438 - precision: 0.9749 - recall: 0.9617 - val_loss: 0.2072 - val_accuracy: 0.9237 - val_precision: 0.9644 - val_recall: 0.9530

Epoch 00003: val_loss improved from 0.

In [9]:
best_model = keras.models.load_model(best_model_path,
    custom_objects= {
        'precision': metrics.precision,
        'recall': metrics.recall
    }
)
validation_result = best_model.evaluate_generator(
    generator=dataset_generators.validate
)

## Results

### Epoch CSV

In [10]:
csv_file_path = f'{paths.results_location}{network_name}/{dataset}-history.csv'
export_learning_history_to_csv(csv_file_path, fit_result, time_callback.times)

epoch, time, val_loss, val_accuracy, val_precision, val_recall
0,0,0,0,0,0
1,2.257840871810913,0.4050242006778717,0.8733813166618347,0.9909090399742126,0.9363636374473572
2,1.1624770164489746,0.23142491281032562,0.9165467619895935,0.9545454382896423,0.8890151381492615
3,1.269618272781372,0.20721451938152313,0.9237409830093384,0.9643939733505249,0.9530303478240967
4,1.1121461391448975,0.17545576393604279,0.9237409830093384,0.8977272510528564,0.8901514410972595
5,1.3057489395141602,0.1536155641078949,0.935251772403717,1.0,0.9621211886405945
6,1.1476960182189941,0.021232236176729202,0.9424460530281067,1.0,0.9886363744735718
7,1.2114360332489014,0.023720787838101387,0.9338129758834839,0.9772727489471436,0.9829545617103577
8,1.2960147857666016,0.08673059195280075,0.9237409830093384,0.8636363744735718,0.8192640542984009
9,1.376729965209961,0.21580463647842407,0.9410071969032288,0.9734848141670227,0.9909090399742126
10,1.1685900688171387,2.5519893824821338e-05,0.9525179862976074,0.95454543828

### Validation result

In [11]:
csv_file_path = f'{paths.results_location}{network_name}/{dataset}-result.csv'
export_learning_validation_result_to_csv(csv_file_path, validation_result, time_callback.times)

avg_epoch_time, time, val_loss, val_accuracy, val_precision, val_recall
1.2138982343673705,60.69491171836853,0.1866678148508072,0.9512894153594971,0.9772727489471436,1.0
